In [21]:
import numpy as np
import pickle
import torch
import timeit
from tqdm.autonotebook import trange

In [2]:
#The mechanisms are presorted so path finding is not needed here, Also the time needed for path finding is minimal in comparison to the actual solution.
file = open("10KM.pkl", 'rb')
data = pickle.load(file)
file.close()

In [3]:
def preprocess_data(data):
    Cs = np.zeros([len(data[0]),20,20])
    nt = np.zeros([len(data[0]),20])
    x0s = np.zeros([len(data[0]),20,2])
    
    for i in range(len(data[0])):
        C = np.array(data[0][i])
        x0 = np.array(data[1][i])
        fn = data[2][i]
        Cs[i,0:C.shape[0],0:C.shape[0]] = C
        x0s[i,0:C.shape[0],:] = x0
        nt[i,fn] = 1
        nt[i,C.shape[0]:] = 1
        
    return Cs,np.expand_dims(nt,-1),x0s

In [4]:
As,node_types,x0s = preprocess_data(data)

In [8]:
def solve_rev_vectorized_batch(As,x0s,node_types,thetas):
    
    Gs = torch.cdist(x0s,x0s)
    
    x = torch.zeros([x0s.shape[0],x0s.shape[1],thetas.shape[0],2]).to('cuda')
    
    x = x + torch.unsqueeze(node_types * x0s,2)
    
    m = x[:,0] + torch.tile(torch.unsqueeze(torch.transpose(torch.cat([torch.unsqueeze(torch.cos(thetas),0),torch.unsqueeze(torch.sin(thetas),0)],0),0,1),0),[x0s.shape[0],1,1]) * torch.unsqueeze(torch.unsqueeze(Gs[:,0,1],-1),-1)
    
    x[:,1,:,:] = m
    
    for k in range(3,x0s.shape[1]):
        
        inds = torch.argsort(As[:,k,0:k])[:,-2:]
        
        l_ijs = torch.linalg.norm(x[np.arange(x0s.shape[0]),inds[:,0]] - x[np.arange(x0s.shape[0]),inds[:,1]], dim=-1)
        
        gik = torch.unsqueeze(Gs[np.arange(x0s.shape[0]),inds[:,0],np.ones(shape=[x0s.shape[0]])*k],-1)
        gjk = torch.unsqueeze(Gs[np.arange(x0s.shape[0]),inds[:,1],np.ones(shape=[x0s.shape[0]])*k],-1)
        
        cosphis = (torch.square(l_ijs) + torch.square(gik) - torch.square(gjk))/(2 * l_ijs * gik)
        
        cosphis = torch.where(torch.tile(node_types[:,k],[1,thetas.shape[0]])==0.0,cosphis,torch.zeros_like(cosphis))
                             
        x0i1 = x0s[np.arange(x0s.shape[0]),inds[:,0],np.ones(shape=[x0s.shape[0]]).astype(np.int32)]
        x0i0 = x0s[np.arange(x0s.shape[0]),inds[:,0],np.zeros(shape=[x0s.shape[0]]).astype(np.int32)]
        
        x0j1 = x0s[np.arange(x0s.shape[0]),inds[:,1],np.ones(shape=[x0s.shape[0]]).astype(np.int32)]
        x0j0 = x0s[np.arange(x0s.shape[0]),inds[:,1],np.zeros(shape=[x0s.shape[0]]).astype(np.int32)]
        
        x0k1 = x0s[:,k,1]
        x0k0 = x0s[:,k,0]
        
        s = torch.unsqueeze(torch.sign((x0i1-x0k1)*(x0i0-x0j0) - (x0i1-x0j1)*(x0i0-x0k0)),-1)
        

        phi = s * torch.arccos(cosphis)
        
        a = torch.permute(torch.cat([torch.unsqueeze(torch.cos(phi),0),torch.unsqueeze(-torch.sin(phi),0)],0),dims=[1,2,0])
        b = torch.permute(torch.cat([torch.unsqueeze(torch.sin(phi),0),torch.unsqueeze(torch.cos(phi),0)],0),dims=[1,2,0])

        R = torch.einsum("ijk...->jki...", torch.cat([torch.unsqueeze(a,0),torch.unsqueeze(b,0)],0))
        
        xi = x[np.arange(x0s.shape[0]),inds[:,0]]
        xj = x[np.arange(x0s.shape[0]),inds[:,1]]
        
        scaled_ij = (xj-xi)/torch.unsqueeze(l_ijs,-1) * torch.unsqueeze(gik,-1)
        
        x_k = torch.squeeze(torch.matmul(R, torch.unsqueeze(scaled_ij,-1))) + xi
        x_k = torch.where(torch.tile(torch.unsqueeze(node_types[:,k],-1),[1,thetas.shape[0],2])==0.0,x_k,torch.zeros_like(x_k))
        x[:,k,:,:] += x_k
    return x

In [9]:
As = torch.Tensor(As).to('cuda')
node_types = torch.Tensor(node_types).to('cuda')
x0s = torch.Tensor(x0s).to('cuda')
thetas = torch.Tensor(np.linspace(0,np.pi*2,201)[0:200]).to('cuda')

In [24]:
timer = trange(100)
for i in timer:
    solve_rev_vectorized_batch(As,x0s,node_types,thetas)

  0%|          | 0/100 [00:00<?, ?it/s]

In [28]:
1/timer.format_dict['rate']

0.11235209151374764